In [47]:
import os
import pandas as pd
from numpy.random import normal

os.chdir('/Users/nikkivanhandel/CS6400/mfgdb/CS6400-MfgDB/')
path = '//Users/nikkivanhandel/Downloads/NIST_24/NIST_23/XYPT Commands/'

The following data set is from the NIST Additive Manufacturing Metrology Testbed. The data comprises 250 layers in a single build. A selection of layers is used for demonstration purposes 

https://www.nist.gov/publications/process-monitoring-dataset-additive-manufacturing-metrology-testbed-ammt-overhang-part

In the original file, absolute time is not given. Rather, each row corresponds to a command given at 100 kHz; each row is 1 microsecond apart. These values are what the machine is "told" to do rather than waht it actually does. 

 While the values for true position and  files for this build are available, the records are very large and difficult to acquire. Instead, we use the programmed data and add noise to treat it as collected data. To use this for the example, we will downsample to data to 100 Hz and add normally distributed noise approximated to the machinery accuracy. 

In [49]:
import sqlite3

conn = sqlite3.connect("mfgdb")
cursor = conn.cursor()

# get the next index position, i.e. where to add data from
q = "SELECT MAX(RecordID) FROM SensorData"
try:
    cursor.execute(q)
    conn.commit()
    v = cursor.fetchall()[0][0]
    print(v)

except sqlite3.Error as e:
    print(f"An error occurred: {e}")
    conn.rollback()

conn.close()

156206


In [80]:
# Channel names (x position, yposition, power, trigger (laser on or off))
start_time = pd.to_datetime('12-06-2024 14:00')  # Time that build starts
dwell = pd.to_timedelta(5, unit='s') # time between layers
start_id = v
freq = 100000
ds_rate = 2000 # 

layer_start  = start_time
for i in range(1,6):
    file = os.path.join(path, f'L000{i}.csv')
    df = pd.read_csv(file, names=['X', 'Y', 'P', 'T']) 
    df.drop('T', axis=1, inplace=True) # Trigger command isn't imporant 
    # Downsample from 100,000 Hz to 50 Hz, each time incrememnt is 0.01s apart
    down = df.loc[::ds_rate].reset_index(drop=True)
    k = len(down)
    down['time'] = pd.date_range(start=layer_start,
                                 end=layer_start + pd.to_timedelta(k*ds_rate/freq, 
                                                                unit='s'),
                                 periods=k)
    down['date'] = down['time'].dt.date
    down['time'] = down['time'].dt.time

    # Apply noise to signals based on whats reasonable for the measurement
    down.X  = down.X + normal(0, 0.01, k)
    down.Y = down.Y + normal(0, 0.01, k)
    down.P = down.P + normal(0.1, k)
    
    layer_start += dwell
    break

In [ ]:
for chan in ['X', 'Y', 'P']:
    channel = down[:, [chan, '']]

,X,Y,P,time,date
0,-0.022621,0.011729,231.092194,14:00:00,2024-12-06
1,-1.743537,3.176809,231.092194,14:00:00.020101,2024-12-06
2,-3.650978,6.685050,231.092194,14:00:00.040203,2024-12-06
3,-5.558544,10.190424,231.092194,14:00:00.060304,2024-12-06
4,-3.758367,16.008586,331.092194,14:00:00.080406,2024-12-06
...,...,...,...,...,...
193,5.360592,-15.990678,231.092194,14:00:03.879593,2024-12-06
194,4.615302,-13.806852,231.092194,14:00:03.899695,2024-12-06
195,3.335846,-10.001958,231.092194,14:00:03.919796,2024-12-06
196,2.075643,-6.201788,231.092194,14:00:03.939898,2024-12-06
